<a href="https://colab.research.google.com/github/HatefulRock/Chatbot/blob/main/LLama2RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf


In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes


In [ ]:
!pip install install sentence_transformers


In [ ]:
!pip3 uninstall llama-index
#!pip3 install llama-index --upgrade --no-cache-dir --force-reinstall
!pip install llama_index[langchain]

Found existing installation: llama-index 0.10.17
Uninstalling llama-index-0.10.17:
  Would remove:
    /usr/local/bin/llamaindex-cli
    /usr/local/lib/python3.10/dist-packages/llama_index-0.10.17.dist-info/*
    /usr/local/lib/python3.10/dist-packages/llama_index/_bundle/*
Proceed (Y/n)? Y
  Successfully uninstalled llama-index-0.10.17
  Using cached llama_index-0.10.17-py3-none-any.whl (5.6 kB)


In [ ]:
#!pip install llama-index-llms-huggingface
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface  import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='6558e3af-8ff2-4894-b985-7b2ed7f4f098', embedding=None, metadata={'page_label': '1', 'file_name': '/content/data/Les congés maternité et paternité.pdf', 'file_path': '/content/data/Les congés maternité et paternité.pdf', 'file_type': 'application/pdf', 'file_size': 195958, 'creation_date': '2024-03-07', 'last_modified_date': '2024-03-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Les congés maternité et paternité\nLE CONGE MATERNITE\nLes congés de maternité sont également indemnisés par la Sécurité Sociale. Cette prestation est, dans ce cas aussi, \ncomplétée par l’établissement pour que le salaire reste le même. C’est à partir de la date présumée de l’accouchement \nque sont comptabilisés les jours de congés.\nLeur durée normale est

In [ ]:
system_prompt="""
Vous êtes un assistant questions-réponses. Votre objectif est de répondre aux questions comme
avec autant de précision que possible en fonction des instructions et du contexte fournis.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-65e97ad6-162b1cfb5c7ed8253c3de0c4;a8ee4f0e-2e06-4bb5-ac54-acf5b06063b1)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-chat-hf to ask for access.

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [ ]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
service_context


In [ ]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)


In [ ]:
index


In [ ]:
query_engine=index.as_query_engine()


In [ ]:
response=query_engine.query("what is attention is all you need?")


In [ ]:
print(response)


In [ ]:
response=query_engine.query("what is YOLO?")


In [ ]:
print(response)
